In [2]:
from __future__ import print_function

import os
import numpy as np
from skimage import filters
from skimage.feature import corner_peaks
import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import HTML
from skimage import img_as_float
from skimage.io import imread
from matplotlib import animation

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

In [7]:
def load_frames(imgs_dir):
    frames = [img_as_float(imread(os.path.join(imgs_dir, frame), 
                                               as_grey=True)) \
              for frame in sorted(os.listdir(imgs_dir))]

    fig,ax=plt.subplots(figsize=(10,8))
    ax.axis('off')
    im = ax.imshow(frames[0])
    
    def animate(i):
        im.set_array(frames[i])
        return [im,]      
    
    ani = animation.FuncAnimation(fig,animate,frames=len(frames),interval=50,blit=True)
    return ani

ani = load_frames('imgs')
HTML(ani.to_html5_video())                                                                                            
                                                                                             

ValueError: Could not find a format to read the specified file in mode 'i'

# 0. pre-processing 

using high-pass(guassian) filter to make an image appear sharper.


## 1.1 lucas-kanade method for optical flow, 

the equation of the  intensity of one point between two consecutive pictures$ I(x,y)=I(x +\Delta x,y+\Delta y,t+\Delta t)$
 which is equal to the derivative to t: $\frac{\partial I}{\partial x} v_{x}+\frac{\partial I}{\partial y} v_{y}+\frac{\partial I}{\partial t} =0$
 
 we apply this for each pixel:$I_{x}\left(p_{i}\right) v_{x}+I_{y}\left(p_{i}\right) v_{y}=-I_{t}\left(p_{i}\right)$
 
 $$
A=\left[\begin{array}{cc}
I_{x}\left(p_{1}\right) & I_{y}\left(p_{1}\right) \\
I_{x}\left(p_{2}\right) & I_{y}\left(p_{2}\right) \\
\vdots & \vdots \\
I_{x}\left(p_{n}\right) & I_{y}\left(p_{n}\right)
\end{array}\right] \quad v=\left[\begin{array}{c}
v_{x} \\
v_{y}
\end{array}\right] \quad b=\left[\begin{array}{c}
-I_{t}\left(p_{1}\right) \\
-I_{t}\left(p_{2}\right) \\
\vdots \\
-I_{t}\left(p_{n}\right)
\end{array}\right]
$$
We can now solve for the flow vectors (now represented as $v$ ) by solving the following least-square problem:

$A^{T} A v=A^{T} b$.


## 1.2 harris corner detector using



In [11]:
from skimage import filters
from skimage.feature import corner_harris, corner_peaks

keypoints = corner_peaks(corner_harris(frames[0]),
                       exclude_border=10,
                       threshold_rel=0.005)
plt.figure(figsize=(10,12))
plt.imshow(frames[0])
plt.scatter(keypoints[:,1],keypoints[:,0],facecolor='none',edgecolors='r')
plt.axis('off')
plt.show()

NameError: name 'frames' is not defined

In [ ]:
def lucas_kanade(im1,im2,keypoints,window_size=5):
    assert window_size %2==1,'window size must be an odd number'
    flow_vectors=[]
    w=window_size//2
    
    Iy,Ix=np.gradient(im1)
    It=im2-im1
    for y,x in keypoints:
        y, x =int(round(y)),int(round(x))
        A1 = Ix[y-w: y+w+1, x-w: x+w+1]
        A2 = Iy[y-w: y+w+1, x-w: x+w+1]
        A = np.c_[A1.reshape(-1,1), A2.reshape(-1,1)]
        b = -It[y-w: y+w+1, x-w: x+w+1].reshape(-1,1)
        d = np.dot(np.linalg.inv(A.T.dot(A)), A.T.dot(b))
        flow_vectors.append(d.flatten())
    flow_vectors=np.array(flow_vectors)
    #code
    
    return flow_vectors

flow_vectors=lucas_kanade(frames[0],frames[1],keypoints,window_size=5)
plt.figure(figsize=(15,12))
plt.imshow(frames[0])
plt.axis('off')
plt.title('optical flow')

for y,x,vy,vx in np.hstack((keypoints,flow_vectors)):
    plt.arrow(x,y,vx,vy,head_width=3,head_length=5,color='y')

## 1.3 iterative Lucas-Kanande method 

p is a pixel on frame I, the goal is to find the flow vector(velocity):

$v =[v_{x}  \quad v_{y} ]^{T}$  such that p+v is the corresponding point on the next frame J


1.3.1 Initialize flow vector:
$$
v=\left[\begin{array}{l}
0 \\
0
\end{array}\right]
$$
1.3.2 Compute spatial gradient matrix:
$$
G =(A^{T}A)\\
G=\sum_{x=p_{x}-w}^{p_{x}+w} \sum_{y=p_{y}-w}^{p_{y}+w}\left[\begin{array}{cc}
I_{x}^{2}(x, y) & I_{x}(x, y) I_{y}(x, y) \\
I_{x}(x, y) I_{y}(x, y) & I_{y}^{2}(x, y)
\end{array}\right]\\
$$

for $k=1$ to $K$

1.3.3 Compute temporal difference: $\delta I_{k}(x, y)=I(x, y)-J\left(x+g_{x}+v_{x}, y+g_{y}+v_{y}\right)$


1.3.4 Compute image mismatch vector:

$$
b_{k}= A^{T}b\\
\\
b_{k}=\sum_{x=p_{x}-w}^{p_{x}+w} \sum_{y=p_{y}-w}^{p_{y}+w}\left[\begin{array}{l}
\delta I_{k}(x, y) I_{x}(x, y) \\
\delta I_{k}(x, y) I_{y}(x, y)
\end{array}\right]\\
$$
1.3.5 Compute optical flow: $v^{k}=G^{-1} b_{k}$

Update flow vector for next iteration: $v:=v+v^{k}$

Return $v$  


In [10]:
def iterative_lucas_kanade((im1, im2, keypoints,
                            window_size=9,
                            num_iters=7,g=None):
        if g is None:
        g = np.zeros(keypoints.shape)

        flow_vectors = []
        w = window_size // 2
        Iy,Ix =np.gradient(im1)
        It=img2-img1
                           
        for y,x,gy,gx in np.hstack((keypoints,g)):
            v=np.zeros(2)
            yi, xi = int(round(y)), int(round(x))
            
            A1=Ix[yi-w:yi+w+1, xi-w:xi+w+1]
            A2=Iy[yi-w:yi+w+1, xi-w:xi+w+1]   
            G = np.array([[np.sum(A1**2), np.sum(A1*A2)], [np.sum(A1*A2), np.sum(A2**2)]])
            invG = np.linalg.inv(G)               
        flow_vectors = np.array(flow_vectors)

    return flow_vectors   
                           
                       
                           
# Plot flow vectors
plt.figure(figsize=(15,12))
plt.imshow(frames[0])
plt.axis('off')
plt.title('Optical flow vectors (iterative LK)')

for y, x, vy, vx in np.hstack((keypoints, flow_vectors)):
    plt.arrow(x, y, vx, vy, head_width=5, head_length=5, color='b')

SyntaxError: invalid syntax (<ipython-input-10-84e80d639884>, line 1)